## Schema dictionaries

The basis of the input/output parsers is a set of functions, which extract type, order and various other information from the `FleurInputSchema.xsd` and `FleurOutputSchema.xsd` files for different versions. The obtained information is stored in large dictionaries next to the schema files.

To load the information we use the `InputSchemaDict` and `OutputSchemaDict` classes by providing the desired version string to the `fromVersion` classmethod. They both work in the same way but are needed, since the outputschema implicitly includes information from the inputschema.

These classes not only contain small helper functions but are also immutable, i.e. trying to modify the parsed information will raise an error.

In [ ]:
from masci_tools.io.parsers.fleur.fleur_schema import InputSchemaDict
from pprint import pprint
help(InputSchemaDict)

schema_dict = InputSchemaDict.fromVersion('0.33')

The returned python object also contains a xmlschema object for validating files

In [ ]:
print(type(schema_dict.xmlschema))

Let's for example take a look at `attrib_types`. Here all attributes are classified for the conversion from the strings we get from the xml file. If there are multiple possible types the conversion function will start at the first type and stop when a conversion was successful (`string` is put in last place at all times)

In [ ]:
pprint(schema_dict['attrib_types'])

In `tag_paths` all possible names of tags are mapped to possible simple xpaths through the input file

In [ ]:
pprint(schema_dict['tag_paths'])

There are multiple keys for attributes and text tags (`unique_attribs`, `unique_path_attribs` and `other_attribs`), which classify the attributes in terms of three categories:

1. unique attributes can only occur once in the input file and there is only one possible path
2. unique path attributes can occur in multiple places but each place has only one occurence (name clashes for exmaple `spinf`)
3. other attributes

In [ ]:
pprint(schema_dict['unique_attribs'])

In [ ]:
pprint(schema_dict['unique_path_attribs'])

This is useful but it does not provide a utility, to get a path and guarantee that you end up with a unique path. For this there are the functions `get_tag_xpath` and `get_attrib_xpath`. They are used by providing the name of the tag/attribute in question and other criteria to select the right path

In [ ]:
print(schema_dict.get_tag_xpath('bzIntegration'))

If the path is not unique an error is raised and we have to be more specific with the selection

In [ ]:
print(schema_dict.get_tag_xpath('ldaU'))

In [ ]:
print(schema_dict.get_tag_xpath('ldaU', contains='species'))

In [ ]:
print(schema_dict.get_tag_xpath('ldaU', not_contains='atom'))

If there is no possible path to fullfill the criteria the function also raises an error

In [ ]:
print(schema_dict.get_tag_xpath('ldaU', contains='species', not_contains='atom'))

These functions allow for easy version support between different file versions if the tag names themselves do not change

In [ ]:
schema_dict_max4 = InputSchemaDict.fromVersion('0.31')
print(schema_dict.get_attrib_xpath('valenceElectrons'))
print(schema_dict_max4.get_attrib_xpath('valenceElectrons'))

More detailed information about the attributes and tags, which can be on a given tag can be found in the `tag_info` key. This part of the schema dictionary is indexed by the simple xpaths to avoid name clashes

In [ ]:
pprint(schema_dict['tag_info']['/fleurInput'])

In [ ]:
pprint(schema_dict['tag_info']['/fleurInput/atomSpecies/species'])